# Paraphrase Classification Model

## Data Loading

Load the training and validation data into dataframes

In [1]:
import pandas as pd

train_df = pd.read_csv('./data/paws/train.tsv', sep='\t')[['sentence1', 'sentence2', 'label']]
val_df = pd.read_csv('./data/paws/dev.tsv', sep='\t')[['sentence1', 'sentence2', 'label']]
train_df.head()


,sentence1,sentence2,label
0,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1


Use the dataframes to create huggingface datasets

In [2]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

## Tokenize Data

Load the tokenizer of the model that will be fine-tuned

In [3]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = 'sentence-transformers/all-distilroberta-v1'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

This tokenize function tokenizes both sentences and concatenates them with a seperator token

In [4]:
def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True)

Apply the tokenize function to each dataset

In [5]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

100%|██████████| 8/8 [00:00<00:00,  8.04ba/s]


## Load model

Load the huggingface model for sequence classification with 2 labels

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to('cuda')

Some weights of the model checkpoint at sentence-transformers/all-distilroberta-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-distilroberta-v1 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN t

## Get Trainer Config

Load the accuracy and f1 metrics and create a function that applies them to pass to the trainer

In [7]:
import numpy as np
from datasets import load_metric

metric = load_metric('accuracy', 'f1')

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Create the TrainingArguments. This specifies checkpoint path, batch_size, epochs, when to apply the metrics, etc.

In [8]:
from transformers import TrainingArguments
batch_size = 64
training_args = TrainingArguments('./data/models/test_paraphrase_distilbert_1', per_device_train_batch_size=batch_size, per_device_eval_batch_size=batch_size, num_train_epochs=5, evaluation_strategy='epoch', report_to="wandb")

# Train the model
Create the Trainer with everything created so far and run the train function.

In [9]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
train_output = trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, sentence2.
***** Running training *****
  Num examples = 49401
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3860
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: phillipmerritt (use `wandb login --relogin` to force relogin)
c:\VSCodeDrive\5290_text_summarization\env\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidg

 13%|█▎        | 500/3860 [02:24<21:34,  2.60it/s]Saving model checkpoint to ./data/models/test_paraphrase_distilbert_1\checkpoint-500
Configuration saved in ./data/models/test_paraphrase_distilbert_1\checkpoint-500\config.json


{'loss': 0.6317, 'learning_rate': 4.352331606217617e-05, 'epoch': 0.65}


Model weights saved in ./data/models/test_paraphrase_distilbert_1\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./data/models/test_paraphrase_distilbert_1\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./data/models/test_paraphrase_distilbert_1\checkpoint-500\special_tokens_map.json
 20%|██        | 772/3860 [03:39<13:23,  3.84it/s]The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, sentence2.
***** Running Evaluation *****
  Num examples = 8000
  Batch size = 64

 20%|██        | 772/3860 [03:50<13:23,  3.84it/s]

{'eval_loss': 0.3568389415740967, 'eval_accuracy': 0.8465, 'eval_runtime': 11.853, 'eval_samples_per_second': 674.934, 'eval_steps_per_second': 10.546, 'epoch': 1.0}


...

{'eval_loss': 0.3860807418823242, 'eval_accuracy': 0.904875, 'eval_runtime': 11.8483, 'eval_samples_per_second': 675.204, 'eval_steps_per_second': 10.55, 'epoch': 4.0}


 91%|█████████ | 3500/3860 [16:44<02:20,  2.56it/s]Saving model checkpoint to ./data/models/test_paraphrase_distilbert_1\checkpoint-3500
Configuration saved in ./data/models/test_paraphrase_distilbert_1\checkpoint-3500\config.json


{'loss': 0.0863, 'learning_rate': 4.663212435233161e-06, 'epoch': 4.53}


Model weights saved in ./data/models/test_paraphrase_distilbert_1\checkpoint-3500\pytorch_model.bin
tokenizer config file saved in ./data/models/test_paraphrase_distilbert_1\checkpoint-3500\tokenizer_config.json
Special tokens file saved in ./data/models/test_paraphrase_distilbert_1\checkpoint-3500\special_tokens_map.json
100%|██████████| 3860/3860 [18:23<00:00,  3.84it/s]The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence1, sentence2.
***** Running Evaluation *****
  Num examples = 8000
  Batch size = 64

100%|██████████| 3860/3860 [18:35<00:00,  3.84it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 3860/3860 [18:35<00:00,  3.46it/s]

{'eval_loss': 0.38545864820480347, 'eval_accuracy': 0.90675, 'eval_runtime': 11.8056, 'eval_samples_per_second': 677.642, 'eval_steps_per_second': 10.588, 'epoch': 5.0}
{'train_runtime': 1123.8149, 'train_samples_per_second': 219.792, 'train_steps_per_second': 3.435, 'train_loss': 0.23392994935030764, 'epoch': 5.0}


In [10]:
train_output

TrainOutput(global_step=3860, training_loss=0.22098883544842815, metrics={'train_runtime': 1111.6416, 'train_samples_per_second': 222.198, 'train_steps_per_second': 3.472, 'train_loss': 0.22098883544842815, 'epoch': 5.0})